## Demo for Property Inference Attack (PIA)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/privML/privacy-evaluator/blob/team2sprint4/notebooks/property_inference_attack.ipynb) [![Open in Github](https://raw.githubusercontent.com/privML/privacy-evaluator/main/notebooks/images/GitHub-Mark-32px.png)](https://github.com/privML/privacy-evaluator/blob/team2sprint4/notebooks/property_inference_attack.ipynb)

In [7]:
!pip3 install git+https://github.com/privML/privacy-evaluator@team2sprint4

com/privML/privacy-evaluator (to revision team2sprint4) to /private/var/folders/_b/r90tkt1x6tvfxwwbrkgtkbpc0000gn/T/pip-req-build-fj7df9jr
  Running command git clone -q https://github.com/privML/privacy-evaluator /private/var/folders/_b/r90tkt1x6tvfxwwbrkgtkbpc0000gn/T/pip-req-build-fj7df9jr
  Running command git checkout -b team2sprint4 --track origin/team2sprint4
  Switched to a new branch 'team2sprint4'
  Branch 'team2sprint4' set up to track remote branch 'team2sprint4' from 'origin'.



# Property Inference Attack on MNIST Dataset

## 1. Overview of the Dataset

MNIST is a dataset of black-and-white handwritten digits from 10 classes (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), consisting of 60000 training- and 10000 test-images.

The size of each image is $28\times 28 \times 1$:



In [1]:
from privacy_evaluator.attacks.property_inference_attack import PropertyInferenceAttack
from privacy_evaluator.classifiers.classifier import Classifier
from privacy_evaluator.utils.data_utils import (
    dataset_downloader,
    new_dataset_from_size_dict,
)
from privacy_evaluator.utils.trainer import trainer
from privacy_evaluator.models.torch.cnn import ConvNet

import collections
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")



In [2]:
train_dataset, test_dataset = dataset_downloader("MNIST")
input_shape = test_dataset[0][0].shape
print(input_shape)

(28, 28, 1)


As of now we can only perform binary attacks. Therefore as an example we use classes 0 and 1 from MNIST.

We choose a sample size for each class (on which we train the target model):

In [3]:
NUM_ELEMENTS_PER_CLASSES = {0: 1000, 1: 500}

And then adjust the training set accordingly:

In [4]:
train_set = new_dataset_from_size_dict(train_dataset, NUM_ELEMENTS_PER_CLASSES)
print(train_set[0].shape, train_set[1].shape)

(1500, 28, 28, 1) (1500,)


## 2. Load and train your target model

Load *your* personal target model in the next cell to perform the attack on it. Any PyTorch or TensorFlow model can be used.


In [5]:
# For demonstration purposes we load the following ConvNet model for two input classes
# (multi-class input will be supported in future releases):

num_classes = len(NUM_ELEMENTS_PER_CLASSES)

model = ConvNet(num_classes, input_shape, num_channels=(input_shape[-1], 16, 32, 64))  ### <--- Customize this line

In [6]:
trainer(train_set, NUM_ELEMENTS_PER_CLASSES, model, num_epochs=8)

# Convert to ART classifier

target_model = Classifier._to_art_classifier(model, "sparse_categorical_crossentropy", num_classes, input_shape)

## 3. Perform attack

Each attack consists of several sub-attack (one for each element in "ratios_for_attack").

In a sub-attack we create a number of shadow classifiers of the same architecture as the provided target model. 

Half of them will be trained on an unbalanced data set of the given ratio, the other half is trained on blanced data sets.

The shadow classifiers serve as training set for a meta classifier, which finally predicts the likelihood of the target model to have a given property (i.e. the ratio).

In [7]:
# Number of shadow classifiers (increase for better accuracy of the meta classifier, decrease when not enough computing power is available.)
amount_sets = 2 #needs to be even

# Size of data set to train the shadow classifiers
size_set = 6
#Ratios to perform the attack for (the real ratios of our example target model is 0.66: {0: 1000, 1: 500}: 66% of data points are from class 0, 33% from class 1.)
ratios_for_attack = [0.66,0.33]
classes = [0,1]

attack = PropertyInferenceAttack(target_model, train_set, verbose=1, size_set=size_set, \
    ratios_for_attack=ratios_for_attack, classes=classes,amount_sets=amount_sets)

In [8]:
output = attack.attack()

Initiating Property Inference Attack ... 
Extracting features from target model ... 
(97634,)  --- features extracted from the target model.
Creating set of 1 balanced shadow classifiers ... 
Creating shadow training sets
Training shadow classifiers
Performing PIA for various ratios ... 
  0%|          | 0/2 [00:00<?, ?it/s]Creating shadow training sets
Training shadow classifiers
Epoch 1/2
1/1 [==============================] - 4s 4s/step - loss: 0.6164 - accuracy: 1.0000
Epoch 2/2
 50%|█████     | 1/2 [00:04<00:04,  4.43s/it]Creating shadow training sets
Training shadow classifiers
Epoch 1/2
1/1 [==============================] - 4s 4s/step - loss: 0.8347 - accuracy: 0.0000e+00
Epoch 2/2
100%|██████████| 2/2 [00:08<00:00,  4.42s/it]


In [10]:
output.output

{'class 0: 0.67, class 1: 0.33': 0.5026385,
 'class 0: 0.34, class 1: 0.66': 0.4971639}

## Human readable output of the attack:

In [12]:
output.max_message

'The most probable property is class 0: 0.67, class 1: 0.33 with a probability of 0.5026385188102722.'